# Advent of Code 2015

## Day 19: Medicine for Rudolph (PART 2 UNFINISHED)

Solution code by [leechristie](https://github.com/leechristie) for Advent of Code 2015.

In [1]:
import random
import time


def load_input(filename: str) -> tuple[list[tuple[str, str]], str]:
    with open(filename) as file:
        replacements = []
        for line in file:
            line = line.strip()
            if not line:
                break
            line = line.split(' => ')
            replacements.append(tuple(line))
        final = None
        for line in file:
            assert not final
            final = line
        return replacements, final.strip('\r\n ')

REPLACEMENTS, MOLECULE = load_input('data/input19.txt')

In [2]:
REPLACEMENTS

[('Al', 'ThF'),
 ('Al', 'ThRnFAr'),
 ('B', 'BCa'),
 ('B', 'TiB'),
 ('B', 'TiRnFAr'),
 ('Ca', 'CaCa'),
 ('Ca', 'PB'),
 ('Ca', 'PRnFAr'),
 ('Ca', 'SiRnFYFAr'),
 ('Ca', 'SiRnMgAr'),
 ('Ca', 'SiTh'),
 ('F', 'CaF'),
 ('F', 'PMg'),
 ('F', 'SiAl'),
 ('H', 'CRnAlAr'),
 ('H', 'CRnFYFYFAr'),
 ('H', 'CRnFYMgAr'),
 ('H', 'CRnMgYFAr'),
 ('H', 'HCa'),
 ('H', 'NRnFYFAr'),
 ('H', 'NRnMgAr'),
 ('H', 'NTh'),
 ('H', 'OB'),
 ('H', 'ORnFAr'),
 ('Mg', 'BF'),
 ('Mg', 'TiMg'),
 ('N', 'CRnFAr'),
 ('N', 'HSi'),
 ('O', 'CRnFYFAr'),
 ('O', 'CRnMgAr'),
 ('O', 'HP'),
 ('O', 'NRnFAr'),
 ('O', 'OTi'),
 ('P', 'CaP'),
 ('P', 'PTi'),
 ('P', 'SiRnFAr'),
 ('Si', 'CaSi'),
 ('Th', 'ThCa'),
 ('Ti', 'BP'),
 ('Ti', 'TiTi'),
 ('e', 'HF'),
 ('e', 'NAl'),
 ('e', 'OMg')]

### Part 1

In [3]:
from collections import defaultdict

REPLACEMENTS1 = defaultdict(list)
REPLACEMENTS2 = defaultdict(list)
for from_element, to_molecule in REPLACEMENTS:
    if len(from_element) == 1:
        REPLACEMENTS1[from_element].append(to_molecule)
    elif len(from_element) == 2:
        REPLACEMENTS2[from_element].append(to_molecule)
    else:
        raise ValueError(f'bad element length: {from_element}')
REPLACEMENTS1 = dict(REPLACEMENTS1)
REPLACEMENTS2 = dict(REPLACEMENTS2)

In [4]:
def possible_new_molecules(molecule, replacements1, replacements2):

    rv = set()

    for index in range(len(molecule)):

        pre = molecule[:index]
        substr1 = molecule[index]
        post1 = molecule[index+1:]
        if substr1 in replacements1:
            for replacement in replacements1[substr1]:
                rv.add(pre + replacement + post1)

        substr2 = molecule[index:index+2]
        if len(substr2) == 2:
            post2 = molecule[index+2:]
            if substr2 in replacements2:
                for replacement in replacements2[substr2]:
                    rv.add(pre + replacement + post2)

    return rv

In [5]:
result = possible_new_molecules(MOLECULE, REPLACEMENTS1, REPLACEMENTS2)

print(f'The number of possible new molecules after 1 replacement is {len(result)}.')

The number of possible new molecules after 1 replacement is 518.


### Part 2

Some encoding stuff.

In [6]:
MUTABLE_ELEMENTS = set()

for k, v in REPLACEMENTS:
    if k != 'e':
        if len(k) == 1:
            MUTABLE_ELEMENTS.add(k)
        else:
            MUTABLE_ELEMENTS.add(k)
MUTABLE_ELEMENTS = sorted(MUTABLE_ELEMENTS)
MUTABLE_ELEMENTS = ['e'] + MUTABLE_ELEMENTS

LEFTOVERS = set()
for _, v in REPLACEMENTS:
    for e in MUTABLE_ELEMENTS:
        v = v.replace(e, ' ')
    v = v.strip()
    if v:
        LEFTOVERS.add(v)

# manually input immutable elements
IMMUTABLE_ELEMENTS = ['CRn', 'Rn', 'Ar', 'Y']

INDEX_TO_ELEMENT =  MUTABLE_ELEMENTS + IMMUTABLE_ELEMENTS
ELEMENT_TO_INDEX = {}
for i, e in enumerate(INDEX_TO_ELEMENT):
    ELEMENT_TO_INDEX[e] = i

def encode(molecule: str) -> tuple[int]:
    rv = []
    while len(molecule) > 0:
        modified = False
        for i, e in enumerate(INDEX_TO_ELEMENT):
            if not modified:
                if molecule.startswith(e):
                    molecule = molecule[len(e):]
                    rv.append(i)
                    modified = True
    return tuple(rv)

def decode(encoding: tuple[int]) -> str:
    rv = ''
    for e in encoding:
        rv += INDEX_TO_ELEMENT[e]
    return rv

Encode the start and end.

In [7]:
ENCODED_START_POINT = encode('e')
assert decode(ENCODED_START_POINT) == 'e'

ENCODED_END_POINT = encode(MOLECULE)
assert decode(ENCODED_END_POINT) == MOLECULE

Encode the replacements forward and backward.

In [8]:
ENCODED_FORWARD_REPLACEMENTS = {}
ENCODED_BACKWARD_REPLACEMENTS = {}

for from_element, to_molecule in REPLACEMENTS:
    encoded_from_element = encode(from_element)
    assert len(encoded_from_element) == 1
    encoded_from_element ,= encoded_from_element
    encoded_to_molecule = encode(to_molecule)
    if encoded_from_element not in ENCODED_FORWARD_REPLACEMENTS:
        ENCODED_FORWARD_REPLACEMENTS[encoded_from_element] = set()
    ENCODED_FORWARD_REPLACEMENTS[encoded_from_element].add(encoded_to_molecule)
    if encoded_to_molecule not in ENCODED_BACKWARD_REPLACEMENTS:
        ENCODED_BACKWARD_REPLACEMENTS[encoded_to_molecule] = set()
    ENCODED_BACKWARD_REPLACEMENTS[encoded_to_molecule].add(encoded_from_element)

for k, v in list(ENCODED_BACKWARD_REPLACEMENTS.items()):
    assert len(v) == 1
    ENCODED_BACKWARD_REPLACEMENTS[k] = list(v)[0]

In [9]:
ENCODED_FORWARD_REPLACEMENTS

{1: {(11, 4), (11, 14, 4, 15)},
 2: {(2, 3), (12, 2), (12, 14, 4, 15)},
 3: {(3, 3),
  (9, 2),
  (9, 14, 4, 15),
  (10, 11),
  (10, 14, 4, 16, 4, 15),
  (10, 14, 6, 15)},
 4: {(3, 4), (9, 6), (10, 1)},
 5: {(5, 3),
  (7, 11),
  (7, 14, 4, 16, 4, 15),
  (7, 14, 6, 15),
  (8, 2),
  (8, 14, 4, 15),
  (13, 1, 15),
  (13, 4, 16, 4, 16, 4, 15),
  (13, 4, 16, 6, 15),
  (13, 6, 16, 4, 15)},
 6: {(2, 4), (12, 6)},
 7: {(5, 10), (13, 4, 15)},
 8: {(5, 9), (7, 14, 4, 15), (8, 12), (13, 4, 16, 4, 15), (13, 6, 15)},
 9: {(3, 9), (9, 12), (10, 14, 4, 15)},
 10: {(3, 10)},
 11: {(11, 3)},
 12: {(2, 9), (12, 12)},
 0: {(5, 4), (7, 1), (8, 6)}}

In [10]:
ENCODED_BACKWARD_REPLACEMENTS

{(11, 4): 1,
 (11, 14, 4, 15): 1,
 (2, 3): 2,
 (12, 2): 2,
 (12, 14, 4, 15): 2,
 (3, 3): 3,
 (9, 2): 3,
 (9, 14, 4, 15): 3,
 (10, 14, 4, 16, 4, 15): 3,
 (10, 14, 6, 15): 3,
 (10, 11): 3,
 (3, 4): 4,
 (9, 6): 4,
 (10, 1): 4,
 (13, 1, 15): 5,
 (13, 4, 16, 4, 16, 4, 15): 5,
 (13, 4, 16, 6, 15): 5,
 (13, 6, 16, 4, 15): 5,
 (5, 3): 5,
 (7, 14, 4, 16, 4, 15): 5,
 (7, 14, 6, 15): 5,
 (7, 11): 5,
 (8, 2): 5,
 (8, 14, 4, 15): 5,
 (2, 4): 6,
 (12, 6): 6,
 (13, 4, 15): 7,
 (5, 10): 7,
 (13, 4, 16, 4, 15): 8,
 (13, 6, 15): 8,
 (5, 9): 8,
 (7, 14, 4, 15): 8,
 (8, 12): 8,
 (3, 9): 9,
 (9, 12): 9,
 (10, 14, 4, 15): 9,
 (3, 10): 10,
 (11, 3): 11,
 (2, 9): 12,
 (12, 12): 12,
 (5, 4): 0,
 (7, 1): 0,
 (8, 6): 0}

In [11]:
def has_collision(set_1: set[tuple[int]], set_2: set[tuple[int]]) -> bool:
    for x in set_1:
        if x in set_2:
            return True
    return False


def advance_forward(current_forward_depth: int,
                    current_forward_lower_or_equal_depth_strings: set[tuple[int]],
                    current_forward_string: set[tuple[int]]) -> int:
    print(f'advancing forward from depth {current_forward_depth} to depth {current_forward_depth + 1}, front size is {len(current_forward_string)}')
    new_forward_string = set()
    for current in current_forward_string:
        for index, value in enumerate(current):
            if value in ENCODED_FORWARD_REPLACEMENTS:
                for replacement in ENCODED_FORWARD_REPLACEMENTS[value]:
                    new_value = current[:index] + replacement + current[index+1:]
                    #print(f'replace {value} at index {index} of {current} with {replacement} to make {new_value}')
                    if new_value not in current_forward_lower_or_equal_depth_strings:
                        current_forward_lower_or_equal_depth_strings.add(new_value)
                        new_forward_string.add(new_value)
    current_forward_string.clear()
    current_forward_string.update(new_forward_string)
    return current_forward_depth + 1


def advance_backward(current_backward_depth: int,
                     current_backward_lower_or_equal_depth_strings: set[tuple[int]],
                     current_backward_string: set[tuple[int]]) -> int:
    print(f'advancing backward from depth {current_backward_depth} to depth {current_backward_depth + 1}, front size is {len(current_backward_string)}')
    new_backward_string = set()
    for current in current_backward_string:
        for index in range(len(current)):
            for match, replacement in ENCODED_BACKWARD_REPLACEMENTS.items():
                if current[index:index+len(match)] == match:
                    new_value = current[:index] + (replacement, ) + current[index+len(match):]
                    #print(f'replace {match} at index {index} of {current} with {replacement} to make {new_value}')
                    if new_value not in current_backward_lower_or_equal_depth_strings:
                        current_backward_lower_or_equal_depth_strings.add(new_value)
                        new_backward_string.add(new_value)
    current_backward_string.clear()
    current_backward_string.update(new_backward_string)
    return current_backward_depth + 1

In [12]:
def solve_part_2(encoded_start_point: tuple[int], encoded_end_point: tuple[int]) -> int:

    print(f'start point is = {encoded_start_point}')
    print(f'end point is = {encoded_end_point}')

    current_forward_depth = 0
    current_backward_depth = 0

    # check if already solved
    if encoded_start_point == encoded_end_point:
        return current_forward_depth + current_backward_depth # will be 0 if doing nothing

    current_forward_lower_depth_strings = set() # non yet by definition, we are at zero
    current_forward_string = {encoded_start_point} # only one for now

    current_backward_lower_depth_strings = set() # non yet by definition, we are at zero
    current_backward_string = {encoded_end_point} # only one for now

    # since ENCODED_START_POINT != ENCODED_END_POINT, we can't have collision yet
    assert not has_collision(current_forward_string, current_backward_string)

    while not has_collision(current_forward_string, current_backward_string):

        # forward depth is less (or equal), advance forward
        if current_forward_depth <= current_backward_depth:

            current_forward_depth = advance_forward(current_forward_depth, current_forward_lower_depth_strings, current_forward_string)

        # backward depth is less, advance backward
        else:

            current_backward_depth = advance_backward(current_backward_depth, current_backward_lower_depth_strings, current_backward_string)



solve_part_2(ENCODED_START_POINT, ENCODED_END_POINT)

start point is = (0,)
end point is = (13, 10, 14, 3, 9, 12, 6, 16, 3, 9, 12, 14, 4, 15, 10, 11, 4, 15, 3, 10, 11, 10, 11, 9, 2, 3, 3, 10, 14, 10, 14, 12, 12, 6, 15, 9, 2, 3, 9, 6, 16, 9, 12, 14, 4, 15, 4, 15, 3, 10, 14, 2, 9, 6, 15, 9, 14, 3, 9, 12, 14, 4, 15, 3, 10, 11, 3, 3, 4, 15, 9, 2, 3, 3, 9, 12, 12, 14, 4, 15, 3, 10, 14, 10, 1, 16, 10, 11, 14, 4, 15, 15, 3, 10, 14, 2, 4, 15, 3, 3, 10, 14, 10, 11, 3, 3, 3, 4, 16, 3, 9, 12, 2, 3, 10, 11, 3, 10, 11, 9, 6, 15, 10, 14, 3, 9, 2, 4, 16, 3, 3, 4, 15, 3, 3, 3, 3, 10, 11, 3, 10, 14, 9, 14, 4, 15, 9, 2, 10, 11, 9, 14, 4, 15, 10, 14, 6, 15, 3, 4, 16, 4, 15, 3, 10, 14, 10, 1, 15, 12, 12, 12, 12, 12, 12, 12, 14, 9, 6, 15, 9, 12, 12, 12, 2, 10, 14, 10, 1, 15, 12, 12, 14, 9, 6, 15, 3, 4, 16, 2, 9, 2, 9, 12, 14, 10, 14, 6, 15, 10, 11, 3, 4, 15, 3, 10, 11, 4, 15, 9, 14, 4, 15, 3, 10, 14, 12, 2, 10, 11, 10, 14, 10, 1, 16, 3, 4, 15, 9, 14, 4, 15, 10, 11, 3, 4, 15, 3, 3, 10, 11, 3, 3, 3, 10, 14, 9, 14, 3, 4, 15, 4, 16, 9, 6, 15, 3, 9, 2, 3, 9, 2, 10

KeyboardInterrupt: 

In [ ]:
print(f'start point is = {ENCODED_START_POINT}')

current_forward_depth = 0

current_forward_lower_depth_strings = set() # non yet by definition, we are at zero
current_forward_string = {ENCODED_START_POINT} # only one for now

while True:

    current_forward_depth = advance_forward(current_forward_depth, current_forward_lower_depth_strings, current_forward_string)

Can go about 10 steps forward from start and about 4 steps backward from end.

In [24]:
for k, v in ENCODED_FORWARD_REPLACEMENTS.items():
    for vv in v:
        vvv = ', '.join([str(vvvv) for vvvv in vv])
        c = '{'
        d = '}'
        print(f'addForward({k}, new int[] {c}{vvv}{d})')

addForward(1, new int[] {11, 14, 4, 15})
addForward(1, new int[] {11, 4})
addForward(2, new int[] {2, 3})
addForward(2, new int[] {12, 14, 4, 15})
addForward(2, new int[] {12, 2})
addForward(3, new int[] {10, 11})
addForward(3, new int[] {10, 14, 4, 16, 4, 15})
addForward(3, new int[] {9, 2})
addForward(3, new int[] {3, 3})
addForward(3, new int[] {10, 14, 6, 15})
addForward(3, new int[] {9, 14, 4, 15})
addForward(4, new int[] {9, 6})
addForward(4, new int[] {10, 1})
addForward(4, new int[] {3, 4})
addForward(5, new int[] {8, 14, 4, 15})
addForward(5, new int[] {13, 4, 16, 6, 15})
addForward(5, new int[] {7, 14, 4, 16, 4, 15})
addForward(5, new int[] {7, 14, 6, 15})
addForward(5, new int[] {13, 6, 16, 4, 15})
addForward(5, new int[] {5, 3})
addForward(5, new int[] {8, 2})
addForward(5, new int[] {7, 11})
addForward(5, new int[] {13, 4, 16, 4, 16, 4, 15})
addForward(5, new int[] {13, 1, 15})
addForward(6, new int[] {12, 6})
addForward(6, new int[] {2, 4})
addForward(7, new int[] {5, 10}

In [28]:
for k, v in ENCODED_BACKWARD_REPLACEMENTS.items():
    kkk = ', '.join([str(kkkk) for kkkk in k])
    c = '{'
    d = '}'
    print(f'addBackward(new int[] {c}{kkk}{d}, {v});')

addBackward(new int[] {11, 4}, 1);
addBackward(new int[] {11, 14, 4, 15}, 1);
addBackward(new int[] {2, 3}, 2);
addBackward(new int[] {12, 2}, 2);
addBackward(new int[] {12, 14, 4, 15}, 2);
addBackward(new int[] {3, 3}, 3);
addBackward(new int[] {9, 2}, 3);
addBackward(new int[] {9, 14, 4, 15}, 3);
addBackward(new int[] {10, 14, 4, 16, 4, 15}, 3);
addBackward(new int[] {10, 14, 6, 15}, 3);
addBackward(new int[] {10, 11}, 3);
addBackward(new int[] {3, 4}, 4);
addBackward(new int[] {9, 6}, 4);
addBackward(new int[] {10, 1}, 4);
addBackward(new int[] {13, 1, 15}, 5);
addBackward(new int[] {13, 4, 16, 4, 16, 4, 15}, 5);
addBackward(new int[] {13, 4, 16, 6, 15}, 5);
addBackward(new int[] {13, 6, 16, 4, 15}, 5);
addBackward(new int[] {5, 3}, 5);
addBackward(new int[] {7, 14, 4, 16, 4, 15}, 5);
addBackward(new int[] {7, 14, 6, 15}, 5);
addBackward(new int[] {7, 11}, 5);
addBackward(new int[] {8, 2}, 5);
addBackward(new int[] {8, 14, 4, 15}, 5);
addBackward(new int[] {2, 4}, 6);
addBackward(ne